In [1]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import lightgbm as lgb

In [2]:
train_df = pd.DataFrame()
folder_path = '../Data_Q1/train'

for i in range(20):  
    file_name = f'data_{i}.csv'
    file_path = os.path.join(folder_path, file_name)

    # 读取CSV文件并合并到all_data
    data = pd.read_csv(file_path)
    train_df = pd.concat([train_df, data])
    print('Successfully read train file: ', file_name)

test_df = pd.read_csv('../Data_Q1/test/test_set.csv')

Successfully read train file:  data_0.csv
Successfully read train file:  data_1.csv
Successfully read train file:  data_2.csv
Successfully read train file:  data_3.csv
Successfully read train file:  data_4.csv
Successfully read train file:  data_5.csv
Successfully read train file:  data_6.csv
Successfully read train file:  data_7.csv
Successfully read train file:  data_8.csv
Successfully read train file:  data_9.csv
Successfully read train file:  data_10.csv
Successfully read train file:  data_11.csv
Successfully read train file:  data_12.csv
Successfully read train file:  data_13.csv
Successfully read train file:  data_14.csv
Successfully read train file:  data_15.csv
Successfully read train file:  data_16.csv
Successfully read train file:  data_17.csv
Successfully read train file:  data_18.csv
Successfully read train file:  data_19.csv


In [3]:
print(train_df.shape)
train_df.head()

(134229, 8)


,x,y,z,a,b,c,d,Is_Falling
0,18.495860,13.766527,14.362624,0,0,0,1,0
1,18.501072,13.827225,14.270268,0,0,1,0,0
2,18.405950,13.868976,14.094804,1,0,0,0,0
3,18.444572,13.910701,14.116078,0,1,0,0,0
4,18.418470,13.933917,14.320566,0,0,0,1,0


In [4]:
print(test_df.shape)
test_df.head()

(6623, 9)


,ID,x,y,z,a,b,c,d,Is_Falling
0,1,6.912997,11.518698,15.471855,0,0,0,1,0
1,2,6.936432,11.574586,15.446939,0,0,1,0,0
2,3,6.935274,11.571790,15.437505,1,0,0,0,0
3,4,6.886688,11.561593,15.704019,0,0,0,1,0
4,5,6.921823,11.597728,15.634435,0,0,1,0,0


In [5]:
# 预处理
X_train = train_df.drop('Is_Falling', axis=1)
y_train = train_df['Is_Falling']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 处理数据不平衡
smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X_train_scaled, y_train)

# 准备测试数据
X_test = test_df.drop(['ID', 'Is_Falling'], axis=1)
y_test = test_df['Is_Falling']
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 训练 RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_sm, y_sm)

# 预测与评估
rf_predictions = rf_model.predict(X_test_scaled)
print("Random Forest Classifier Evaluation")
print(classification_report(y_test, rf_predictions))

In [ ]:
# 训练 LightGBM Classifier
lgbm_model = lgb.LGBMClassifier()
lgbm_model.fit(X_sm, y_sm)

# 预测与评估
lgbm_predictions = lgbm_model.predict(X_test_scaled)
print("LightGBM Classifier Evaluation")
print(classification_report(y_test, lgbm_predictions))


In [8]:
# 创建输出 DataFrame
# 此时需要排除 'Is_Falling' 列，保留 'ID' 列
output_df = pd.DataFrame({'ID': test_df['ID'], 'Is_Falling': lgbm_predictions})
output_df.to_csv('../Q1_output.csv', index=False)